In [1]:
import os
import cv2
import numpy as np

In [2]:
def get_image(directory):
    Image = []
    Labels = []
    label = 0
    
    for labels in os.listdir(directory):
        if labels == 'ka':
            label = 0
        elif labels == 'ca':
            label = 1
        elif labels == 'ta':
            label = 2
        elif labels == 'pa':
            label = 3
        elif labels == 'ya':
            label = 4
        elif labels == 'wa':
            label = 5
        elif labels == 'ga':
            label = 6
        elif labels == 'ja':
            label = 7
        elif labels == 'da':
            label = 8
        elif labels == 'ba':
            label = 9
        elif labels == 'ra':
            label = 10
        elif labels == 'sa':
            label = 11
        elif labels == 'nga':
            label = 12
        elif labels == 'nya':
            label = 13
        elif labels == 'na':
            label = 14
        elif labels == 'ma':
            label = 15
        elif labels == 'la':
            label = 16
        elif labels== 'ha':
            label = 17
            
        for image_file in os.listdir(directory+labels):
            image = cv2.imread(directory+labels+r'/'+image_file)
            image = cv2.resize(image,(75,75))
            
            Image.append(image)
            Labels.append(label)
            
    return Image, Labels

def get_classlabel(class_code):
    labels = {0:'ka', 1:'ca', 3:'pa', 4:'ya', 5:'wa', 6:'ga', 7:'ja', 8:'da',
              9:'ba', 10:'ra', 11:'sa', 12:'nga', 13:'nya', 14:'na', 15:'ma',
              16:'la', 17:'ha'}
    return labels[class_code]


In [4]:
Images, Labels = get_image('train/')

x_train = np.array(Images)
y_train = np.array(Labels)

In [5]:
Images, Labels = get_image('test/')

x_test = np.array(Images).astype('float32')
y_test = np.array(Labels)

In [6]:
from keras.utils import to_categorical
train_Y_one_hot = to_categorical(y_train)
test_Y_one_hot = to_categorical(y_test)

Using TensorFlow backend.


In [7]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import Adam

local_weights_file = 'pretrained/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(input_shape = (75, 75, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
     layer.trainable = False
        
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 75, 75, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 37, 37, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 37, 37, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 37, 37, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
import tensorflow as tf
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.959):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

In [1]:
from tensorflow.keras.optimizers import RMSprop
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.5)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (18, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])

NameError: name 'pre_trained_model' is not defined

In [2]:
history=model.fit(x_train,train_Y_one_hot,epochs=10,validation_data=(x_test,test_Y_one_hot))

NameError: name 'model' is not defined

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
pred_images,no_labels = get_image('pred/')
pred_images = np.array(pred_images).astype('float32')
pred_images.shape

In [ ]:
from keras.preprocessing import image
path = 'D:/Belajar/skripsi/ngalegena/pred/aPredic/nga.10.jpg'
img = image.load_img(path, target_size=(75, 75))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=1)
images = np.vstack([x])
classes = model.predict(images, batch_size=10)
print(np.argmax(classes[0]))


In [ ]:
x = pred_images[199]
imgplot = plt.imshow((x* 255).astype(np.uint8))
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model.predict(images, batch_size=10)
print(classes[0])

In [13]:
predictions = model.predict(x_test)
predictions[400]

array([1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 1.5665216e-20, 0.0000000e+00, 8.2537899e-14,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00], dtype=float32)

In [ ]:
from random import randint
import matplotlib.pyplot as plt
f,ax = plt.subplots(5,5) 
f.subplots_adjust(0,0,3,3)
y = np.argmax(train_Y_one_hot,axis=1)
for i in range(0,5,1):
    for j in range(0,5,1):
        rnd_number = randint(0,len(Images))
        ax[i,j].imshow(x_train[rnd_number])
        ax[i,j].set_title(get_classlabel(y[rnd_number]))
        ax[i,j].axis('off')

In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from random import randint
fig = plt.figure(figsize=(30, 30))
outer = gridspec.GridSpec(5, 5, wspace=0.2, hspace=0.2)

for i in range(25):
    inner = gridspec.GridSpecFromSubplotSpec(2, 1,subplot_spec=outer[i], wspace=0.1, hspace=0.1)
    rnd_number = randint(0,len(pred_images))
    pred_image = np.array([pred_images[rnd_number]])
    x = np.argmax(model.predict(pred_image)[0])
    pred_class = get_classlabel(x)
    for j in range(2):
        if (j%2) == 0:
            ax = plt.Subplot(fig, inner[j])
            ax.imshow((pred_image[0]*255).astype(np.uint8))
            ax.set_title(pred_class)
            ax.set_xticks([])
            ax.set_yticks([])
            fig.add_subplot(ax)


fig.show()

In [ ]:
from keras.models import load_model
from keras.preprocessing.image import img_to_array, load_img
import numpy as np

img_width, img_height = 32, 32

frist_try = 'D:/Belajar/skripsi/aksarasunda/second_try.h5'
test_model = load_model(frist_try)
img = load_img('D:/Belajar/skripsi/aksarasunda/1.jpg',False,target_size=(img_width,img_height))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
preds = test_model.predict_classes(x)
prob = test_model.predict_proba(x)
print(preds, prob)

In [ ]:
import pandas as pd
test_filenames = os.listdir("pred/aPredic")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, load_img

test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    "pred/aPredic", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=(75,75),
    batch_size=10,
    shuffle=False
)

In [ ]:
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/10))

In [ ]:
test_df['category'] = np.argmax(predict, axis=-1)

In [ ]:
label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)

In [ ]:
np.argmax(classes[0],axis=1)

In [ ]:
filenames = os.listdir("pred/aPredic")
categories = []
for filename in filenames:
    labels = filename.split('.')[0]
    if labels == 'ka':
        categories.append(0)
    elif labels == 'ca':
        categories.append(1)
    elif labels == 'ta':
        categories.append(2)
    elif labels == 'pa':
        categories.append(3)
    elif labels == 'ya':
        categories.append(4)
    elif labels == 'wa':
        categories.append(5)
    elif labels == 'ga':
        categories.append(6)
    elif labels == 'ja':
        categories.append(7)
    elif labels == 'da':
        categories.append(8)
    elif labels == 'ba':
        categories.append(9)
    elif labels == 'ra':
        categories.append(10)
    elif labels == 'sa':
        categories.append(11)
    elif labels == 'nga':
        categories.append(12)
    elif labels == 'nya':
        categories.append(13)
    elif labels == 'na':
        categories.append(14)
    elif labels == 'ma':
        categories.append(15)
    elif labels == 'la':
        categories.append(16)
    elif labels== 'ha':
        categories.append(17)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [ ]:
df['category'].value_counts().plot.bar()